# 7 Visualise activity in a video.

We have extracted all the features we plan to use. Overlaying these on the video was useful.
But watching annotated videos is inefficient and not always informative.. 

To help with understanding we build a few tools that let's see at a glance what happens over time.

# 7.1 Plot movements over time. 

In each frame let's find the `centre of gravity` for each person (the average of all the high-confidence marker points). This is handy for time series visualisation. For example plotting the cog.x for each person over time shows how they move closer and further from each other. 

In [ ]:
import os
import utils
import calcs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
videos_in = r"..\LookitLaughter.test"
data_out = r"..\data\1_interim"
videos_out = r"..\data\2_final"

#a couple of files for testing
VIDEO_FILE  = os.path.join(videos_in, "2UWdXP.joke1.rep2.take1.Peekaboo.mp4")
VIDEO_FILE2 = os.path.join(videos_in, "2UWdXP.joke2.rep1.take1.NomNomNom.mp4")
AUDIO_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.wav")
SPEECH_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.json")

testset = [VIDEO_FILE, VIDEO_FILE2] 

processedvideos = utils.getprocessedvideos(data_out)
processedvideos.head()

In [ ]:
keypoints = utils.readKeyPointsFromCSV(processedvideos,VIDEO_FILE)


In [ ]:
#this bit of pandas magic calculates average x and y for all the rows.

keypoints[["cogx","cogy"]] = keypoints.apply(lambda row: calcs.rowcogs(row.iloc[8:59]), axis=1, result_type='expand')

In [ ]:
child = keypoints[keypoints["person"]=="child"]
adult = keypoints[keypoints["person"]=="adult"]

#a plot of child's centre of gravity frame by frame
childplot = plt.plot(child["frame"], child["cogx"], c="red", alpha=0.5)
## add line of adult's centre of gravity
adultplot = plt.plot(adult["frame"], adult["cogx"], c="blue", alpha=0.5)
#add legend
plt.legend(['child', 'adult'], loc='upper left')

In [ ]:
#out of curiousity, let's plot the location of the adult and child's nose

childnoseplot = plt.plot(child["frame"], child["nose.x"], c="red", alpha=0.5)
adultnoseplot = plt.plot(adult["frame"], adult["nose.x"], c="blue", alpha=0.5)
plt.legend(['child', 'adult'], loc='upper left')

# 7.2 TODO Use Voxel51 and PytorchVideo for examining videos

Voxel51 seems to be a useful tool for looking at training data (and trained predictions).

Let's start with the minimal 

https://docs.voxel51.com/user_guide/dataset_creation/index.html


[fiftyone-examples/examples/pytorchvideo_tutorial.ipynb at master · voxel51/fiftyone-examples (github.com)](https://github.com/voxel51/fiftyone-examples/blob/master/examples/pytorchvideo_tutorial.ipynb)



In [ ]:
import fiftyone as fo


# Create a dataset from a directory of videos
dataset = fo.Dataset.from_videos_dir("../LookitLaughter.test")

# Launch the App
session = fo.launch_app(dataset)

# 7.3 Let's normalise all the coordinates

Let's convert all coordinates to values between 0 and 1. But let's preserve aspect ratios. 
So divide all x and y coords by the max of (height, width). This also has the advantage of being easier to code.
